<a href="https://colab.research.google.com/github/Nanangk/Dicoding_Course_Pengembangan_Machine_Learning/blob/master/Multiclass_Text_Classification_with_SLTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')

In [13]:
df.head()

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [14]:
#drop judul_film karena tidak diperlukan
df = df.drop(columns=['judul_film'])

In [15]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [16]:
#change genre to categorical
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [17]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [18]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
#lakukan tokenisasi
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)

In [21]:
#konversi sampel menjadi sequence
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

In [22]:
#samakan paddingnya
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [23]:
#buat model
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
#lakukan training
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 26s - loss: 1.6102 - accuracy: 0.1803 - val_loss: 1.6101 - val_accuracy: 0.1940
Epoch 2/30
26/26 - 29s - loss: 1.6057 - accuracy: 0.2463 - val_loss: 1.6069 - val_accuracy: 0.2289
Epoch 3/30
26/26 - 28s - loss: 1.5573 - accuracy: 0.3595 - val_loss: 1.5992 - val_accuracy: 0.2139
Epoch 4/30
26/26 - 28s - loss: 1.3138 - accuracy: 0.4490 - val_loss: 1.6408 - val_accuracy: 0.2687
Epoch 5/30
26/26 - 33s - loss: 0.9605 - accuracy: 0.6032 - val_loss: 1.9475 - val_accuracy: 0.2836
Epoch 6/30
26/26 - 30s - loss: 0.6296 - accuracy: 0.7749 - val_loss: 2.0998 - val_accuracy: 0.2886
Epoch 7/30
26/26 - 30s - loss: 0.3592 - accuracy: 0.8930 - val_loss: 3.0787 - val_accuracy: 0.2736
Epoch 8/30
26/26 - 28s - loss: 0.1512 - accuracy: 0.9602 - val_loss: 3.1724 - val_accuracy: 0.2736
Epoch 9/30
26/26 - 29s - loss: 0.0883 - accuracy: 0.9789 - val_loss: 3.8463 - val_accuracy: 0.3184
Epoch 10/30
26/26 - 28s - loss: 0.0571 - accuracy: 0.9838 - val_loss: 4.0844 - val_accuracy: 0.2637
Epoch 11/